<a href="https://colab.research.google.com/github/djliden/numerai_starter_kit/blob/main/Numerai_Starter_Kit_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introduction
This notebook will walk you through the entire process of making a [numerai](numer.ai) submission, from downloading the data to submitting final predictions, all in a google colab notebook. In particular, it will address two challenges:
- handling API keys in a remote environment (colab)
- parsing the large CSV files which, if read all at once, will exceed colab's memory and cause the notebook to crash.

This notebook will implement two models: a basic tabular neural network using `fastai` and a linear regression model using `scikit-learn`.

## 1.1 Installing and Importing Dependencies
First, we install and import the necessary packages. This cell is currently set *not* to print any output; if you run into any issues and need to check for error messages, comment out the `%%capture` line

In [1]:
%%capture
# install
!pip install --upgrade python-dotenv fastai numerapi dask

# import dependencies
import gc
import os
from dotenv import load_dotenv, find_dotenv
import dask.dataframe as dd
from getpass import getpass
import pandas as pd
import numpy as np
import numerapi
from fastai.tabular.all import *
from pathlib import Path
from scipy.stats import spearmanr
import sklearn.linear_model
from tqdm import tqdm

## 1.2 Setting Up numerapi
We will use the [numerapi](https://github.com/uuazed/numerapi) package to access the data and make submissions. For this to work, numerapi needs to use your API keys (which can be obtained [here](https://numer.ai/submit)). We will set up two main ways of passing these API keys to a numerapi instance:
1. Read a `.env` file using the `python-dotenv` package. This will require you to upload a `.env` file (which contains your secret key and should *not* be kept under version control). Using this method means you will not have to directly enter your keys each time you use this notebook, though you will need to re-upload the `.env` file.
2. Manually entering the API keys -- if you don't have access to, or don't want to mess with, your `.env` file.

If you have a `.env` file, upload it to the default working directory, `content`, now. In either case, run the cell below to set up the numerapi instance. See [Appendix A](#app_a) for instructions on generating and downloading a .env file.

In [2]:
# Load the numerapi credentials from .env or prompt for them if not available
def credential():
    dotenv_path = find_dotenv()
    load_dotenv(dotenv_path)

    if os.getenv("NUMERAI_PUBLIC_KEY"):
        print("Loaded Numerai Public Key into Global Environment!")
    else:
        os.environ["NUMERAI_PUBLIC_KEY"] = getpass("Please enter your Numerai Public Key. You can find your key here: https://numer.ai/submit -> ")
    
    if os.getenv("NUMERAI_SECRET_KEY"):
        print("Loaded Numerai Secret Key into Global Environment!")
    else:
        os.environ["NUMERAI_SECRET_KEY"] = getpass("Please enter your Numerai Secret Key. You can find your key here: https://numer.ai/submit -> ")
    
    if os.getenv("NUMERAI_MODEL_ID"):
        print("Loaded Numerai Model ID into Global Environment!")
    else:
        os.environ["NUMERAI_MODEL_ID"] = getpass("Please enter your Numerai Model ID. You can find your key here: https://numer.ai/submit -> ")

credential()
public_key = os.environ.get("NUMERAI_PUBLIC_KEY")
secret_key = os.environ.get("NUMERAI_SECRET_KEY")
model_id = os.environ.get("NUMERAI_MODEL_ID")
napi = numerapi.NumerAPI(verbosity="info", public_id=public_key, secret_key=secret_key)

Loaded Numerai Public Key into Global Environment!
Loaded Numerai Secret Key into Global Environment!
Loaded Numerai Model ID into Global Environment!


You can read up on the functionality of numerapi [here](https://github.com/uuazed/numerapi). You can use it to download the competition data, view other numerai users' public profiles, check submission status, manage your stake, and much more. In this case, we'll only be using it to download competition data and submit predictions.

## 1.3 Downloading Competition Data
In a more structured project, you'll probably want to keep the data in a seprate directory from your scripts etc. You could also link google colab to your google drive and store the data there in order to avoid needing to download and process the data every time. In this case, however, we'll keep everything in `./content`, and download the data fresh each time.

In [3]:
napi.download_current_dataset()

2021-04-05 20:50:55,450 INFO numerapi.base_api: target file already exists


'./numerai_dataset_258.zip'

## 1.4 Generating the Training Sample

If you look at the files we downloaded above, you'll see a `numerai_tournament_data.csv` file and a `numerai_training_data.csv` file. The "tournament" file contains many rows with targets which we can use for validation, so let's extract those and combine them with our training set. Note that this cell saves a new `csv` after combining the training and validation data, so we can avoid the time-consuming parsing process if we run this cell again in the same session.

In [4]:
tourn_file = Path(f'./numerai_dataset_{napi.get_current_round()}/numerai_tournament_data.csv')
train_file = Path(f'./numerai_dataset_{napi.get_current_round()}/numerai_training_data.csv')
processed_pkl = Path('./training_processed.pkl')

def process_current(processed_train_pickle,
                    train_file, tourn_file,
                    pickle_out=True):
    
    if processed_train_pickle.exists():
        print("Loading the pickled training data from file\n")
        training_data = pd.read_pickle(processed_train_pickle)
    else:
        print("Processing training data...\n")
        tourn_iter_csv = dd.read_csv(tourn_file)
        tmp_df = tourn_iter_csv[tourn_iter_csv['data_type'] == 'validation']
        training_data = dd.read_csv(train_file)
        training_data = dd.concat([training_data, tmp_df])
        training_data = training_data.compute()
        training_data.reset_index(drop=True, inplace=True)
        #tourn_iter_csv.close()
        print("Training Dataset Generated! Saving to file ...")
        
        if pickle_out:
            training_data.to_pickle(processed_train_pickle)
        
    feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]
    target_cols = ['target']
    return training_data, feature_cols, target_cols

training_data, feature_cols, target_cols = process_current(processed_pkl,
                                                           train_file,
                                                           tourn_file)

train_idx = training_data.index[training_data.data_type=='train'].tolist()
val_idx = training_data.index[training_data.data_type=='validation'].tolist()

Processing training data...

Training Dataset Generated! Saving to file ...


# 2 Modeling the Data

In this section, we will define our evaluation metrics; run two different models (a linear regression model from `scikit-learn` and a neural network from `fastai`); and generate submission dataframes from those files.

## 2.1 Evaluation Metrics

In this section, we will define two key evaluation metrics used to assess the performance of models before submitting to the tournament. These metrics are:
- Average Spearman Correlation per era: The sum of each era's Spearman correlation divided by the number of eras.
- Sharpe Ratio: The average correlation per era divided by the standard deviation of the correlations per era.

Both are defined in reasonable detail [here](https://wandb.ai/carlolepelaars/numerai_tutorial/reports/How-to-get-Started-With-Numerai--VmlldzoxODU0NTQ). The methods defined below are modified versions of the methods described in that post.

In [5]:
def corr(df: pd.DataFrame) -> np.float32:
    """
    Calculate the correlation by using grouped per-era data
    :param df: A Pandas DataFrame containing the columns "era", "target" and "prediction"
    :return: The average per-era correlations.
    """
    def _score(sub_df: pd.DataFrame) -> np.float32:
        """ Calculate Spearman correlation for Pandas' apply method """
        return spearmanr(sub_df["target"],  sub_df["prediction"])[0]
    corrs = df.groupby("era").apply(_score)
    return corrs.mean() 

def sharpe(df: pd.DataFrame) -> np.float32:
    """
    Calculate the Sharpe ratio by using grouped per-era data
    :param df: A Pandas DataFrame containing the columns "era", "target" and "prediction"
    :return: The Sharpe ratio for your predictions.
    """
    def _score(sub_df: pd.DataFrame) -> np.float32:
        """ Calculate Spearman correlation for Pandas' apply method """
        return spearmanr(sub_df["target"],  sub_df["prediction"])[0]
    corrs = df.groupby("era").apply(_score)
    return corrs.mean() / corrs.std()

## 2.2 Models

### 2.2.1 Linear Regression Model
This model closely follows the tutorial example [here](https://colab.research.google.com/github/numerai/example-scripts/blob/master/making-your-first-submission-on-numerai.ipynb). We will use the `scikit-learn` package, with which we can implement and fit our regression model in just a couple of lines of code.

#### Fitting the Linear Regression Model

In [6]:
train_sample = training_data.iloc[train_idx]
model = sklearn.linear_model.LinearRegression()
model.fit(train_sample[feature_cols], train_sample.target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

#### Assessing Regression Model Performance

Here we apply the `corr` and `sharpe` methods defined above to predictions made on the validation sample in order to estimate our model's tournament performance.

In [7]:
val_sample = training_data.iloc[val_idx]
val_preds = model.predict(val_sample[feature_cols])
eval_df = pd.DataFrame({'prediction':val_preds,
                        'target':val_sample.target,
                        'era':val_sample.era}).reset_index()
val_sharpe = sharpe(eval_df)
val_corr = corr(eval_df)

print((f'The linear regression model\'s validation correlation is {val_corr}. '
       f'Its validation sharpe is {val_sharpe}'))

The linear regression model's validation correlation is 0.016246523230621295. Its validation sharpe is 0.5236836981840626


#### Making Predictions with the Regression Model

In [8]:
ids = []
preds = []

chunksize = 50000

tourn_iter_csv = pd.read_csv(tourn_file, iterator=True, chunksize=1e6)
for chunk in tourn_iter_csv:
    df = chunk[feature_cols]
    out = model.predict(df)
    ids.extend(chunk["id"])
    preds.extend(out)
tourn_iter_csv.close()

In [9]:
linear_regression_predictions_df = pd.DataFrame({
    'id':ids,
    'prediction':preds
})
linear_regression_predictions_df.head()

,id,prediction
0,n0003aa52cab36c2,0.481608
1,n000920ed083903f,0.492837
2,n0038e640522c4a6,0.530817
3,n004ac94a87dc54b,0.497083
4,n0052fe97ea0c05f,0.503089


In [10]:
linear_regression_predictions_df.to_csv("lr_predictions.csv", index=False)

#### Submitting Predictions from the Linear Regression Model

We can use `numerapi` to submit these predictions as follows. Please uncomment the following code block if you'd actually like to make a submission!

In [11]:
# napi.upload_predictions("lr_predictions.csv", model_id=os.environ.get("NUMERAI_MODEL_ID"))

### 2.2.2 Fastai Neural Network
In this section, we use fastai's tabular data processing and modeling capabilities to develop a neural network ([fastai](https://docs.fast.ai/tabular.core.html)).

#### Preparing the Data

We use the [`TabularPandas`](https://docs.fast.ai/tabular.core.html#TabularPandas) class to set up our DataLoaders. We will use a relatively large batch size (`bs=2048`) to facilitate rapid training in this example.

In [12]:
splits = (list(train_idx), list(val_idx))
data = TabularPandas(training_data, cat_names=None,
                    cont_names=list(feature_cols.values),
                    y_names=target_cols, splits = splits)

dls = data.dataloaders(bs = 2048)

#### Set Up and Fit Fastai Model

In this simple example, we will make no attempt to optimize hyperparameters. You can experiment with the batch size, which we set above, along with the number and size of layers, weight decay, and many of the additional configuration options listed [here](https://docs.fast.ai/tabular.model.html#TabularModel).

**Note:** The following cell will run much faster if you are using a colab session with GPU enabled.

In [13]:
learn = tabular_learner(dls, layers=[200,200],
                        loss_func=MSELossFlat(),
                        metrics = [SpearmanCorrCoef()])

learn.fit_one_cycle(3, wd = 0.1)

epoch,train_loss,valid_loss,spearmanr,time
0,0.054009,0.053226,0.004936,00:24
1,0.050633,0.051545,0.007983,00:23
2,0.048986,0.051075,0.010744,00:23


#### Evaluate Performance of Fastai Model

As we did with the regression model, we'll check the validation correlation and sharpe score of this fastai neural network.

In [14]:
prediction, target = learn.get_preds()
prediction = prediction.numpy().squeeze()
target = target.numpy().squeeze()

era = dls.valid_ds.items['era']
eval_df = pd.DataFrame({'prediction':prediction, 'target':target, 'era':era}).reset_index()

val_sharpe = sharpe(eval_df)
val_corr = corr(eval_df)

print((f'The fastai tabular neural network\'s validation correlation is {val_corr}. '
       f'Its validation sharpe is {val_sharpe}'))

The fastai tabular neural network's validation correlation is 0.010995699197097268. Its validation sharpe is 0.5258036230389045


#### Making Predictions with the Fastai Model

There is probably a way to significantly simplify the following cell.

In [15]:
ids = []
preds = []

chunksize = 75000

tourn_iter_csv = pd.read_csv(tourn_file, iterator=True, chunksize=chunksize)
for chunk in tourn_iter_csv:
    chunk.drop(columns = 'target', inplace = True)
    test_dl = dls.test_dl(chunk)
    preds_out,_ = learn.get_preds(dl = test_dl, inner = True)
    preds_out = preds_out.tolist()
    preds_out = [item for sublist in preds_out for item in sublist]
    ids_out = chunk["id"]
    preds.extend(preds_out)
    ids.extend(ids_out)
tourn_iter_csv.close()
#preds_out = [item for sublist in preds for item in sublist]
#ids_out = [item for sublist in ids for item in sublist]

fastai_predictions_df = pd.DataFrame({
'id':ids,
'prediction':preds
})

fastai_predictions_df.head()
fastai_predictions_df.to_csv("fastai_predictions.csv", index=False)

#### Submitting Predictions from the Fastai Model
Please uncomment the following code block if you'd actually like to make a submission!

In [16]:
#napi.upload_predictions("fastai_predictions.csv", model_id=os.environ.get("NUMERAI_MODEL_ID"))

<a name="app_a"></a>
# Appendix A

## Generating and Saving a `.env` file
I recommend filling out this section to generate a `.env` file and then downloading that file for future use. Then, the next time you want to run this notebook, upload the `.env` file and you will not need to enter your credentials manually.

In [ ]:
# Write lines to file

with open('./.env', 'w') as dotenv:
    dotenv.write(f'NUMERAI_PUBLIC_KEY = {getpass("Please enter your Numerai Public Key. You can find your key here: https://numer.ai/submit -> ")}\n')
    dotenv.write(f'NUMERAI_SECRET_KEY = {getpass("Please enter your Numerai Secret Key. You can find your key here: https://numer.ai/submit -> ")}\n')
    dotenv.write(f'NUMERAI_MODEL_ID = {getpass("Please enter your Numerai Model ID. You can find your key here: https://numer.ai/submit -> ")}\n')

To confirm that this worked, you can run `!cat .env` in a new cell and check the output against the values on https://numer.ai/submit. Make sure no one is looking, and that you don't save the output where others on the Internet can find it!

If you want to download this `.env` file for future use (e.g. to re-upload the next time you want to use this notebook), run the next cell:

In [ ]:
from google.colab import files
files.download("./.env")

In a version-controlled project, it is a good practice to add the `.env` file to the project's `.gitignore` so the file is not accessible to others who may download the project from github.